In [1]:
import pandas as pd
from glob import glob
import numpy as np
from scipy.spatial import distance
import re
import pymesh,math

In [ ]:
# !pip install xlrd

In [2]:
data = pd.read_excel("../planilla_colecta.xlsx")

In [3]:
data = data[["id","sexo","peso","promedio altura","promedio cir cad"]]

In [4]:
row_data = data.loc[data['id'] == 1].iloc[0].tolist()

In [5]:
row_data

[1, u'f', 62.299999999999997, 163.40000000000001, 101.045]

In [6]:
fs_landmarks = sorted(glob("../data_artificial_cites/landmarks_reales/*.pts"))
fs_mesh = sorted(glob("./out/torsos/*.ply"))

In [18]:
estatura_error = []
estatura_calculada = []
cadera_calculada=[]
cadera_error=[]
for landmarks_f, mesh_f in zip(fs_landmarks[:], fs_mesh[:]):
    row_data = data.loc[data['id'] == int(re.findall("\d+",landmarks_f)[0])].iloc[0].tolist()
    landmarks = pd.read_csv(landmarks_f,header=None,index_col=[0],skiprows=range(2),delimiter=r"\s+")
    mesh = pymesh.load_mesh(mesh_f)
    #mesh = pymesh.load_mesh(mesh_f)
    head = landmarks.iloc[14].tolist()
    center_foot = np.array([landmarks.iloc[7].tolist(),landmarks.iloc[6].tolist()]).mean(0)
    estatura = distance.pdist(np.array([head,center_foot]))[0]*100
    estatura_calculada.append(estatura)
    estatura_error.append( estatura - row_data[3])
    
    ls0 = landmarks.iloc[0].tolist()
    ls1 = landmarks.iloc[4].tolist()
    ls2 = landmarks.iloc[9].tolist()
    ls3 = landmarks.iloc[10].tolist()
    head = landmarks.iloc[14].tolist()
    foot_left = landmarks.iloc[7].tolist()
    foot_right = landmarks.iloc[6].tolist()
    knee_left = landmarks.iloc[8].tolist()
    knee_right = landmarks.iloc[5].tolist()
    center_knee = np.array([knee_left,knee_right]).mean(0)
    
    def distance3D(a, b):
        return math.sqrt((a[0]-b[0])**2+(a[1]-b[1])**2+(a[2]-b[2])**2)
    
    
    semi_landmark = np.array([ls1,ls2]).mean(0)
    distancias = []
    for v in mesh.vertices:
        distancias.append(distance3D(semi_landmark,v))
    distancias = np.array(distancias)
    centro = mesh.vertices[distancias.argmin()]
    distancias.argmin()
    
    delta = [0.4,0.0003,0.4]
    xdot = (head[0]*center_knee[0] + head[1]*center_knee[1] + head[2]*center_knee[2])
    norma = math.sqrt(head[0]**2+head[1]**2+head[2]**2) * math.sqrt(center_knee[0]**2+center_knee[1]**2+center_knee[2]**2)
    angle_head = (math.cos(xdot/norma) ** -1)
    angle_head_r = math.radians(90 - math.degrees(angle_head))
    q = pymesh.misc.Quaternion(quat=[1,-angle_head_r, 0,0])
    
    

    p1 = centro + delta
    p2 =centro  -  delta
    
    c = pymesh.meshutils.generate_box_mesh(p2,p1 ,1)
    
    alpha = math.pi /8
    #matrix = [[1,0,0],[0,math.cos(alpha),-math.sin(alpha)],[0,math.sin(alpha),math.cos(alpha)],[0,0,0]]
    matrix = [[1,0,0,0],[0,math.cos(alpha),-math.sin(alpha),0.05],[0,math.sin(alpha),math.cos(alpha),0],[0,0,0,1]]

    for i,v in enumerate(c.vertices_ref):
        c.vertices_ref[i] = np.dot(matrix,[v[0],v[1],v[2],1])[:-1] 
        
    output_mesh = pymesh.boolean(c, mesh, operation="intersection",engine="auto")
    vertexs_2d = np.delete(output_mesh.vertices, [1], axis=1)
    
    def distance2D(a, b):
        return math.sqrt( ((b[0]-a[0])**2)  + ((b[1]-a[1])**2) )
    
    mx = np.mean(vertexs_2d[:,0])
    my = np.mean(vertexs_2d[:,1])
    
    pr1 = np.array(sorted(vertexs_2d, key=lambda x : (math.atan2(x[0] - mx, x[1] - my) + 2 * math.pi) % (2*math.pi)))
    aux = 0
    for i,v in enumerate(pr1[:-1]):
        aux = aux + distance2D(v,pr1[i+1])
    aux = aux + distance2D(pr1[0],pr1[-1])    
    cadera_calculada.append(aux*100)
    cadera_error.append((aux*100) - row_data[4])

In [11]:
np.shape(estatura_error)

(5,)

In [13]:
estatura_calculada

[160.66467388320308,
 167.73817356671265,
 168.86697536457967,
 161.78538565915451,
 165.830201341085]

In [15]:
estatura_error

[-2.7353261167969265,
 -1.861826433287348,
 -3.4330246354203382,
 -4.3146143408454805,
 -4.7697986589149934]

In [10]:
data["estatura cal"] = estatura_calculada
data["estatura error"] = estatura_error

ValueError: Length of values does not match length of index

In [ ]:
np.array(cadera_calculada*100)

In [ ]:
data.loc[abs(data['id'])<7]

In [23]:
cadera_calculada

[99.22898470336872,
 112.09238266428292,
 103.10997319645342,
 101.07941945485773,
 100.10038454043185,
 98.75368668376345,
 106.89189768893203,
 95.78312460699587,
 101.78748810895716,
 94.49475912663506,
 105.49722599862281,
 99.67074648075464,
 124.01454675358458,
 103.12396365190244,
 102.9510952783959,
 99.65066763471884,
 97.14318275959172,
 93.73664309244396,
 110.98113244646153,
 117.20413120431192,
 104.6925972744331,
 94.16014608079941,
 96.09246139136879,
 98.37545750017452,
 94.1898076550775,
 109.7657450398713,
 103.74583824166751,
 112.54842087135255,
 98.6040992337247,
 101.95230827654436,
 95.66497075495471,
 94.01910572697612,
 382.2195684577675,
 97.47225658291454,
 102.91258068362869,
 141.40763344807675,
 93.3264503451107,
 99.3328745699544,
 104.35788426726491,
 97.96209226527452,
 111.17806789649154,
 96.59474022328412,
 95.09216675755748,
 100.20244421679917,
 113.05936194321184,
 116.18939976271336,
 117.33751747372854,
 92.50327158243596,
 111.23277511233172,
 

In [24]:
cadera_error

[-1.8160152966312779,
 -0.057617335717083051,
 4.7099731964534186,
 6.1294194548577252,
 -7.1996154595681503,
 -3.3463133162365466,
 1.541897688932039,
 1.8831246069958638,
 4.0874881089571602,
 -2.7052408733649429,
 8.7972259986228067,
 -2.3792535192453528,
 1.1145467535845768,
 3.1739636519024401,
 -5.1989047216041087,
 2.4006676347188431,
 -1.5068172404082816,
 0.88664309244396122,
 -1.418867553538476,
 12.554131204311915,
 1.792597274433092,
 -1.3398539192005927,
 -2.1575386086312136,
 -0.82454249982548333,
 -0.81019234492249836,
 11.265745039871305,
 -1.7541617583324864,
 9.7984208713525476,
 0.0040992337247018895,
 2.8523082765443633,
 2.7649707549547031,
 1.4191057269761274,
 285.21956845776748,
 2.9222565829145424,
 1.6125806836286927,
 -4.1923665519232429,
 -0.57354965488930532,
 6.2328745699544044,
 5.9578842672649017,
 0.46209226527452074,
 1.8780678964915438,
 -3.0052597767158744,
 -1.807833242442527,
 3.2024442167991651,
 6.9093619432118345,
 4.6893997627133643,
 6.3875174

In [21]:
np.mean(cadera_error)

6.3279657914763305

In [22]:
np.std(cadera_error)

40.392196100679612

In [25]:
data["cadera calculada"] = cadera_calculada
data["cadera error"] = cadera_error

In [39]:
data.loc[abs(data['cadera error'])>7]

,id,sexo,peso,promedio altura,promedio cir cad,cadera calculada,cadera error
5,6,m,79.2,170.6,107.30,100.100385,-7.199615
13,14,m,65.9,174.4,96.70,105.497226,8.797226
22,23,m,89.5,187.8,104.65,117.204131,12.554131
28,29,f,65.2,164.4,98.50,109.765745,11.265745
30,31,f,58.0,150.2,102.75,112.548421,9.798421
36,37,f,58.1,156.4,97.00,382.219568,285.219568
54,55,m,80.1,167.4,103.35,95.064512,-8.285488
59,60,f,79.7,163.7,110.25,125.608383,15.358383
63,64,f,56.7,160.4,99.25,108.998560,9.748560
70,71,f,45.8,154.0,82.90,90.571530,7.671530


In [32]:
data.to_csv("primeros_cacl_1.csv")

In [37]:
def reject_outliers(data, m = 2.):
    d = np.abs(data - np.median(data))
    mdev = np.median(d)
    s = d/mdev if mdev else 0.
    print s
    return data[s<m]